In [24]:
from langchain.memory import ConversationSummaryBufferMemory
from langchain.chat_models import ChatOpenAI

llm = ChatOpenAI(temperature=0.1)

memory = ConversationSummaryBufferMemory(
    llm=llm,
    max_token_limit=50,
    return_messages=True
)


def add_message(input, output):
    memory.save_context({"input": input}, {"output": output})


def get_history():
    return memory.load_memory_variables({})


add_message("Hi, My name is Seonghwan, I live in South Korea, Seoul",
            "Cool! How's the weather in Seoul?")

In [25]:
add_message("Seoul is now hot, sweaty sunny day",
            "Be careful to heat, stay inside")

In [26]:
add_message("How for is Seoul to Köln?",
            "It's about 8,566 km far! Super far away!")

In [27]:
add_message("How for is Seoul to QingTao?",
            "It's about 574 km far! It's very short!")

In [28]:
add_message("How for is Seoul to Tokyo?",
            "It's about 1130 km far! It's very short!")

In [29]:
add_message("How for is Seoul to Phuket?",
            "It's about 4375 km far! It's queit long!")

In [30]:
get_history()

{'history': [SystemMessage(content="Seonghwan from South Korea, Seoul, shares that it's a hot, sweaty sunny day in Seoul. The AI advises to be careful of the heat and stay inside. When asked about the distance between Seoul and Köln, the AI responds that it's about 8,566 km far, which is super far away. When asked about the distance between Seoul and QingTao, the AI responds that it's about 574 km far, which is very short in comparison. The human asks how far Seoul is from Tokyo, and the AI responds that it's about 1130 km far, which is very short!"),
  HumanMessage(content='How for is Seoul to Phuket?'),
  AIMessage(content="It's about 4375 km far! It's queit long!")]}